# Post-process predictions for vectorisation

```
#
# Copyright (c) Sinergise, 2019 -- 2021.
#
# This file belongs to subproject "field-delineation" of project NIVA (www.niva4cap.eu).
# All rights reserved.
#
# This source code is licensed under the MIT license found in the LICENSE
# file in the root directory of this source tree.
#
```

This notebook shows how to post-process the predicted extent and boundary pseudo-probabilities to generate a single raster map that will be used for vectorisation.

The following steps are executed:

 * merge predictions over a time interval using percentile statistics
 * join the extent and boundary predictions derived from previous step
 * iteratively smooth and upscale the joined prediction for a visually smoother output
 * export the resulting map as .tif to be used for vectorisation

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)

In [3]:
from pathlib import Path

import geopandas as gpd

from eolearn.core import FeatureType, EOExecutor

from fd.post_processing import (
    get_post_processing_workflow, 
    get_exec_args, 
    PostProcessConfig)

## Paths and config set-up

In [4]:
INPUT_DATA_DIR = Path('../../input-data/')

In [5]:
model_version = 'folds_avg_10e'

post_process_config = PostProcessConfig(
    bucket_name='bucket-name',
    aws_access_key_id='',
    aws_secret_access_key='',
    aws_region='eu-central-1',
    time_intervals=dict(APRIL=(12, 21)), # include last weeks of March and firsts of May
    eopatches_folder='data/Castilla/2020-04/eopatches',
    tiffs_folder='results/Castilla/2020-04/raster',
    feature_extent=(FeatureType.DATA, f'EXTENT_PREDICTED_{model_version}'),
    feature_boundary=(FeatureType.DATA, f'BOUNDARY_PREDICTED_{model_version}'),
    model_version=model_version,
    max_cloud_coverage=0.10 # increased from 0.05 since there aren't good frames available
)

#### List of patches

In [6]:
grid_definition = gpd.read_file(INPUT_DATA_DIR/'cyl-grid-definition.gpkg')
grid_definition.head()

,id,name,geometry
0,2302137,30TVM_2_5,"POLYGON ((-3.96458 41.90790, -3.96594 41.99795..."
1,2301391,30TUN_7_8,"POLYGON ((-4.58294 42.53146, -4.58522 42.62148..."
2,2301392,30TUN_7_9,"POLYGON ((-4.58067 42.44143, -4.58294 42.53146..."
3,2301398,30TUN_8_5,"POLYGON ((-4.46755 42.80315, -4.46969 42.89318..."
4,2301399,30TUN_8_6,"POLYGON ((-4.46543 42.71313, -4.46755 42.80315..."


In [7]:
eopatches_list = grid_definition.name.values

In [8]:
len(eopatches_list)

1083

### Get workflow

In [9]:
workflow = get_post_processing_workflow(post_process_config)

In [10]:
exec_args = get_exec_args(workflow=workflow, 
                          eopatch_list=eopatches_list,
                          config=post_process_config)

In [11]:
len(exec_args)

1083

Multi-process using `EOExecutor`

In [15]:
executor = EOExecutor(workflow, exec_args, save_logs=True)

In [16]:
executor.run(workers=20)

In [17]:
executor.make_report()